In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from re import sub
from decimal import Decimal
from matplotlib import pyplot as plt
import datetime
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, SGDRegressor
from sklearn.preprocessing import Imputer, StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

from math import sqrt
import tests as t
import re


import nltk
from nltk import word_tokenize, WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import folium

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import xgboost as xgb
pd.set_option('display.max_columns', 106)
import statsmodels.api as sm

import collections

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sousa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sousa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sousa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
porto = pd.read_csv(r"C:\Users\sousa\Desktop\github\Airbnb Tale of 2 cities\Data Portugal\porto_listings.csv")
lisbon = pd.read_csv(r"C:\Users\sousa\Desktop\github\Airbnb Tale of 2 cities\Data Portugal\lisbon_listings.csv")

In [3]:
X_porto = porto.drop('price', 1)
y_porto = porto['price']

X_lisbon = lisbon.drop('price', 1)
y_lisbon = lisbon['price']

In [4]:
# creating a dict to save the importance rankings
feature_rank_porto = {}
feature_rank_lisbon = {}

In [5]:
# creating a function to store feature rankings
def feature_rankings (feature_rank, feat_name, order = 1):
    '''
    Takes in the calculated feature ranking and the feature name and returns a dictionary of the two
    Args:
    feature_rank: calculated rank of a feature
    feat_name: the name of the feature
    Returns: a dictionary with the pairs feature ranking and feature name
    '''
    feature_rank = MinMaxScaler().fit_transform(order*np.array([feature_rank]).T).T[0]
    feature_rank = map(lambda x: round(x,2), feature_rank)
    
    return dict(zip(feat_name, feature_rank))

## Feature Selection using RFE

In [6]:
lr = LinearRegression(normalize=True)
lr.fit(X_porto, y_porto)

rfe = RFE(lr, n_features_to_select=1, verbose =3 )
rfe.fit(X_porto, y_porto)

feature_rank_porto["RFE"] = feature_rankings(list(map(float, rfe.ranking_)), X_porto.columns, order=-1)

Fitting estimator with 223 features.
Fitting estimator with 222 features.
Fitting estimator with 221 features.
Fitting estimator with 220 features.
Fitting estimator with 219 features.
Fitting estimator with 218 features.
Fitting estimator with 217 features.
Fitting estimator with 216 features.
Fitting estimator with 215 features.
Fitting estimator with 214 features.
Fitting estimator with 213 features.
Fitting estimator with 212 features.
Fitting estimator with 211 features.
Fitting estimator with 210 features.
Fitting estimator with 209 features.
Fitting estimator with 208 features.
Fitting estimator with 207 features.
Fitting estimator with 206 features.
Fitting estimator with 205 features.
Fitting estimator with 204 features.
Fitting estimator with 203 features.
Fitting estimator with 202 features.
Fitting estimator with 201 features.
Fitting estimator with 200 features.
Fitting estimator with 199 features.
Fitting estimator with 198 features.
Fitting estimator with 197 features.
F

In [7]:
lr.fit(X_lisbon,y_lisbon)

rfe = RFE(lr, n_features_to_select=1, verbose =3 )
rfe.fit(X_lisbon, y_lisbon)

feature_rank_lisbon["RFE"] = feature_rankings(list(map(float, rfe.ranking_)), X_lisbon.columns, order=-1)

Fitting estimator with 224 features.
Fitting estimator with 223 features.
Fitting estimator with 222 features.
Fitting estimator with 221 features.
Fitting estimator with 220 features.
Fitting estimator with 219 features.
Fitting estimator with 218 features.
Fitting estimator with 217 features.
Fitting estimator with 216 features.
Fitting estimator with 215 features.
Fitting estimator with 214 features.
Fitting estimator with 213 features.
Fitting estimator with 212 features.
Fitting estimator with 211 features.
Fitting estimator with 210 features.
Fitting estimator with 209 features.
Fitting estimator with 208 features.
Fitting estimator with 207 features.
Fitting estimator with 206 features.
Fitting estimator with 205 features.
Fitting estimator with 204 features.
Fitting estimator with 203 features.
Fitting estimator with 202 features.
Fitting estimator with 201 features.
Fitting estimator with 200 features.
Fitting estimator with 199 features.
Fitting estimator with 198 features.
F

#Automatic feature selection using Recursive Feature Elimination wrapper method
#Porto
lr = LinearRegression(normalize = True)
lr.fit(X_porto,y_porto)

rfe_porto = RFE(lr, verbose = 4)

rfe_porto = rfe_porto.fit(X_porto,y_porto)

selected_features_porto = pd.DataFrame({'Feature':list(X_porto.columns),'Ranking':rfe_porto.ranking_})
selected_features_porto.sort_values(by='Ranking')

#Automatic feature selection using Recursive Feature Elimination wrapper method
#Lisbon
lr = LinearRegression(normalize = True)
lr.fit(X_lisbon,y_lisbon)

rfe_lisbon = RFE(lr, verbose = 4)

rfe_lisbon = rfe_lisbon.fit(X_lisbon,y_lisbon)

selected_features_lisbon = pd.DataFrame({'Feature':list(X_lisbon.columns),'Ranking':rfe_lisbon.ranking_})
selected_features_lisbon.sort_values(by='Ranking')

porto_rank_1 = selected_features_porto[selected_features_porto.Ranking < 2]
porto_rank_1.head(30)
#list(porto_rank_1.Feature)

porto_rank_2 = selected_features_porto[selected_features_porto.Ranking < 3]
#porto_rank_2.head(80)
list(porto_rank_2.Feature)

## Feature Selection using Linear Regression

In [8]:
lr.fit(X_porto, y_porto)
feature_rank_porto["LinReg"] = feature_rankings(np.abs(lr.coef_), X_porto.columns)

In [9]:
lr.fit(X_lisbon, y_lisbon)
feature_rank_lisbon["LinReg"] = feature_rankings(np.abs(lr.coef_), X_lisbon.columns)

## Feature Selection using Random Forest

In [ ]:
rf = RandomForestRegressor(n_jobs=-1, n_estimators=500, verbose=3)
rf.fit(X_porto, y_porto)
feature_rank_porto["RF"] = feature_rankings(rf.feature_importances_, X_porto.columns)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 500building tree 2 of 500
building tree 3 of 500
building tree 4 of 500
building tree 5 of 500building tree 6 of 500


building tree 7 of 500building tree 8 of 500

building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500

building tree 22 of 500
building tree 23 of 500
building tree 24 of 500


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.6s


building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
building tree 43 of 500
building tree 44 of 500
building tree 45 of 500
building tree 46 of 500building tree 47 of 500
building tree 48 of 500

building tree 49 of 500
building tree 50 of 500
building tree 51 of 500
building tree 52 of 500
building tree 53 of 500
building tree 54 of 500
building tree 55 of 500
building tree 56 of 500
building tree 57 of 500
building tree 58 of 500
building tree 59 of 500
building tree 60 of 500
building tree 61 of 500
building tree 62 of 500
building tree 63 of 500
building tree 64 of 500
building tree 65 of 500
building tree 66

[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    9.0s



building tree 124 of 500
building tree 125 of 500
building tree 126 of 500
building tree 127 of 500
building tree 128 of 500
building tree 129 of 500
building tree 130 of 500
building tree 131 of 500
building tree 132 of 500
building tree 133 of 500
building tree 134 of 500
building tree 135 of 500
building tree 136 of 500
building tree 137 of 500
building tree 138 of 500
building tree 139 of 500
building tree 140 of 500
building tree 141 of 500
building tree 142 of 500
building tree 143 of 500
building tree 144 of 500
building tree 145 of 500
building tree 146 of 500
building tree 147 of 500
building tree 148 of 500
building tree 149 of 500
building tree 150 of 500
building tree 151 of 500


In [ ]:
rf.fit(X_lisbon, y_lisbon)
feature_rank_lisbon["RF"] = feature_rankings(rf.feature_importances_, X_lisbon.columns)

#Porto
rf = RandomForestRegressor(n_estimators = 500, n_jobs=-1, verbose=3)
rf_porto = rf.fit(X_porto,y_porto)


#ranks["RF"] = ranking(rf.feature_importances_, X_porto.columns)
selected_features_porto = pd.DataFrame({'Feature':list(X_porto.columns),'Ranking':ranking(rf.feature_importances_, X_porto.columns)})
selected_features_porto.sort_values(by='Ranking')

#Lisbon
rf = RandomForestRegressor(n_jobs=-1, verbose=3)
rf.fit(X_lisbon,y_lisbon)
ranks["RF"] = ranking(rf.feature_importances_, X_lisbon.columns)

### Displaying the feature selection results Porto

In [ ]:
r_porto = {}

In [ ]:
for col in X_porto.columns:
    r_porto[col] = round(np.mean([feature_rank_porto[method][col] for method in feature_rank_porto.keys()]), 2)
    
methods = sorted(feature_rank_porto.keys())
feature_rank_porto["Mean"] = r_porto
methods.append("Mean")

print("\t%s" % "\t".join(methods))

for col in X_porto.columns:
    print("%s\t%s" % (col, "\t".join(map(str, [feature_rank_porto[method][col] for method in methods]))))

In [ ]:
# Put the mean scores into a Pandas dataframe
meanplot_porto = pd.DataFrame(list(r_porto.items()), columns= ['Feature','Mean_Ranking'])

# Sort the dataframe
meanplot_porto = meanplot.sort_values('Mean_Ranking', ascending=False)

In [ ]:
# Let's plot the ranking of the features
sns.factorplot(x="Mean_Ranking", y="Feature", data = meanplot_porto, kind="bar", 
               size=14, aspect=1.9, palette='coolwarm')

In [ ]:
meanplot_porto.head(50)

In [ ]:
porto_reduced = meanplot_porto.drop(meanplot_porto[(meanplot_porto.Mean_Ranking < 30)].index, axis = 0, inplace = True)

In [ ]:
sns.factorplot(x="Mean_Ranking", y="Feature", data = porto_reduced, kind="bar", 
               size=14, aspect=1.9, palette='coolwarm')

### Displaying the feature selection results Lisbon

In [ ]:
r_lisbon = {}

In [ ]:
for col in X_lisbon.columns:
    r_lisbon[col] = round(np.mean([feature_rank_lisbon[method][col] for method in feature_rank_lisbon.keys()]), 2)
    
methods = sorted(feature_rank_lisbon.keys())
feature_rank_lisbon["Mean"] = r_lisbon
methods.append("Mean")

print("\t%s" % "\t".join(methods))

for col in X_lisbon.columns:
    print("%s\t%s" % (col, "\t".join(map(str, [feature_rank_lisbon[method][col] for method in methods]))))

In [ ]:
# Put the mean scores into a Pandas dataframe
meanplot_lisbon = pd.DataFrame(list(r_lisbon.items()), columns= ['Feature','Mean_Ranking'])

# Sort the dataframe
meanplot_lisbon = meanplot_lisbon.sort_values('Mean_Ranking', ascending=False)

In [ ]:
# Let's plot the ranking of the features
sns.factorplot(x="Mean_Ranking", y="Feature", data = meanplot_lisbon, kind="bar", 
               size=14, aspect=1.9, palette='coolwarm')

In [ ]:
meanplot_lisbon.head(50)

In [ ]:
lisbon_reduced = meanplot_lisbon.drop(meanplot_lisbon[(meanplot_lisbon.Mean_Ranking < 30)].index, axis = 0, inplace = True)

In [ ]:
sns.factorplot(x="Mean_Ranking", y="Feature", data = lisbon_reduced, kind="bar", 
               size=14, aspect=1.9, palette='coolwarm')